## First we import all the necessary libraries

In [1]:
from collections import defaultdict, deque
from operator import itemgetter
import networkx as nx
import statistics
import numpy as np
import time
import pickle

## Importing the given data into data structures that we want

Dictionary of categories:

In [2]:
categories = {}
with open('wiki-topcats-categories.txt', 'r') as f:    
    for i in f:
        i = i.strip('').lstrip('Category:').replace(';','').split()
        if len(i)-1 >= 3500:
            key = i[0]
            value = list(map(int, i[1:]))
            categories.update({key:value})
        else:
            pass

Making the list of page names:

In [3]:
with open('wiki-topcats-page-names.txt', 'r') as f:
    pagenames = [' '.join(i.split()) for i in f]

In [4]:
pagenames[0:10]

['0 Chiasmal syndrome',
 '1 Kleroterion',
 '2 Pinakion',
 '3 LyndonHochschildSerre spectral sequence',
 "4 Zariski's main theorem",
 '5 FultonHansen connectedness theorem',
 "6 Cayley's ruled cubic surface",
 '7 Annulus theorem',
 "8 Bing's recognition theorem",
 '9 BochnerMartinelli formula']

Nested list of connections

In [5]:
with open('wiki-topcats-reduced.txt', 'r') as f:
    connections = [tuple(map(int, i.strip().split())) for i in f]

In [6]:
connections[0:10]

[(52, 401135),
 (52, 1069112),
 (52, 1163551),
 (62, 12162),
 (62, 167659),
 (62, 279122),
 (62, 1089199),
 (62, 1354553),
 (62, 1400636),
 (62, 1403619)]

# Research questions
# RQ1
### Constructing directed graph dictionary

In [7]:
graph = defaultdict(set)
for i in connections:
    graph[i[0]].add(i[1])
    if i[1] not in graph:
        graph[i[1]] = set()    

## Constructing the graph <br>
### We are aware that there is a faster way to build the graph by only one function given by networkX, but we want <br> <br>to add attributes for the nodes, thus we build the graph by normal way.

In [8]:
G = nx.DiGraph()

In [9]:
for key, value in graph.items():
    G.add_node(key)
    for attr in value:
        G.node[key][attr] = pagenames[attr] # insert page name as attribute for each node

In [10]:
for connection in connections:
    G.add_edge(connection[0], connection[1])

### Checking the graph (Answers for RQ1)

In [11]:
nx.info(G)

'Name: \nType: DiGraph\nNumber of nodes: 461193\nNumber of edges: 2645247\nAverage in degree:   5.7357\nAverage out degree:   5.7357'

In [12]:
nx.density(G)

1.2436602635647606e-05

As we can see, the graph is not dense as the density is very low.<br/>
We can also check a node is connected to others and by their names

In [13]:
G.node[62]

{1537692: '1537692 American University',
 12162: '12162 Oliver Sacks',
 1403619: '1403619 James Brady',
 1544420: '1544420 Tim Tebow',
 167659: '167659 Dmitri Nabokov',
 1089199: '1089199 David Eagleman',
 279122: '279122 United States',
 1354553: '1354553 Jay Hambidge',
 1400636: '1400636 Ronald Reagan'}

In [14]:
adj_dict = dict(G.adjacency()) # adjacency dictionary of the graph

## RQ2

In [15]:
# set of articles in the reduced list
set_reduce = set([i for j in connections for i in j])

In [16]:
len(set_reduce)

461193

### Here we only consider those categories with number of connected nodes >= 3500

In [17]:
reduce_categories = {}
for category, articles in categories.items():
    set_articles = set(articles)
    intersect = set_reduce.intersection(set_articles) # only taking the nodes inside the reduce graph
    intersect = list(intersect)
    for article in intersect:
        if len(list(G.neighbors(article))) == 0: # remove the nodes that are not connected
            intersect.remove(article)
    if len(intersect) >= 3500:
        reduce_categories[category] = intersect
        print(category,len(intersect))

English_footballers 6884
The_Football_League_players 7232
Association_football_forwards 4615
Association_football_goalkeepers 3664
Association_football_midfielders 5282
Association_football_defenders 4126
Living_people 323384
Harvard_University_alumni 5262
Major_League_Baseball_pitchers 4803
Members_of_the_United_Kingdom_Parliament_for_English_constituencies 6398
Indian_films 5438
Year_of_death_missing 3629
Year_of_birth_missing_(living_people) 26058
Rivers_of_Romania 7729
Main_Belt_asteroids 11238
Asteroids_named_for_people 4700
English-language_albums 4730
British_films 4382
English-language_films 22358
American_films 15090
People_from_New_York_City 4480
American_television_actors 11409
American_film_actors 13772
Debut_albums 7320
Black-and-white_films 10586
Year_of_birth_missing 3908
Place_of_birth_missing_(living_people) 5092
American_military_personnel_of_World_War_II 3639
Windows_games 3774


In [18]:
len(reduce_categories.keys())

29

In [19]:
giant_set = [] # set of all non-duplicated articles
for category, articles in reduce_categories.items():
    giant_set.extend(articles)
giant_set = list(set(giant_set))
print(len(giant_set))
giant_set[:10]

422144


[1048576,
 1048577,
 1048578,
 1048579,
 1048582,
 1048583,
 1048584,
 1048585,
 1048586,
 1048587]

### Next, we choose "Year_of_death_missing" as the input due to the time factor:

In [20]:
C0 = input() 

Year_of_death_missing


In [21]:
C0_list = reduce_categories.get(C0)
C0_list[0:10]

[1048576,
 1048577,
 1048578,
 1048579,
 1261874,
 720903,
 892935,
 892937,
 892938,
 1048587]

### Self-created bfs for a list

In [22]:
def bfs(adjacency_dict,start,goal_list):
    # dict which holds parents, later helpful to retrieve path.
    # Also useful to keep track of visited node
    parent = defaultdict(lambda:(False, np.inf))

    parent[start] = (True,0)   
    queue = deque([start])
    while queue:
        currNode = queue.popleft()
        for child in adjacency_dict[currNode]:
            if child not in parent: # if not visited
                parent[child] = (True,parent[currNode][1]+1)
                queue.append(child)
    distance_list = []
    for article in goal_list:
        distance_list.append(parent[article][1])
    return distance_list

### Check our function's performance with the default nx.shortest_path_length

nx.shortest_path_length:

In [23]:
Ci_list = reduce_categories.get('Living_people')
category = 'Living_people'
for article in C0_list[:10]:
    bfs_path = []
    for article1 in Ci_list[:10]:
        try:
            distance = nx.shortest_path_length(G,article,article1) # bfs distance
            bfs_path.append(distance)  # a list of shortest paths
        except: # when there is not a path
            bfs_path.append(np.inf)
    print(bfs_path)

[9, 8, inf, inf, 10, 11, 9, 9, inf, 10]
[8, 7, inf, inf, 9, 10, 8, 8, inf, 9]
[7, 6, inf, inf, 8, 9, 7, 7, inf, 8]
[inf, inf, inf, inf, inf, inf, inf, inf, inf, inf]
[6, 5, inf, inf, 6, 7, 6, 6, inf, 6]
[5, 4, inf, inf, 6, 7, 6, 5, inf, 6]
[inf, inf, inf, inf, inf, inf, inf, inf, inf, inf]
[inf, inf, inf, inf, inf, inf, inf, inf, inf, inf]
[inf, inf, inf, inf, inf, inf, inf, inf, inf, inf]
[6, 5, inf, inf, 7, 8, 6, 6, inf, 7]


Our own bfs function:

In [24]:
category = 'Living_people'
Ci_list = reduce_categories.get('Living_people')
for article in C0_list[:10]:
    bfs_path = bfs(adj_dict,article,Ci_list) # bfs distance
    print(bfs_path[0:10])

[9, 8, inf, inf, 10, 11, 9, 9, inf, 10]
[8, 7, inf, inf, 9, 10, 8, 8, inf, 9]
[7, 6, inf, inf, 8, 9, 7, 7, inf, 8]
[inf, inf, inf, inf, inf, inf, inf, inf, inf, inf]
[6, 5, inf, inf, 6, 7, 6, 6, inf, 6]
[5, 4, inf, inf, 6, 7, 6, 5, inf, 6]
[inf, inf, inf, inf, inf, inf, inf, inf, inf, inf]
[inf, inf, inf, inf, inf, inf, inf, inf, inf, inf]
[inf, inf, inf, inf, inf, inf, inf, inf, inf, inf]
[6, 5, inf, inf, 7, 8, 6, 6, inf, 7]


Deleting unnecessary variables to save the memory, as we know that the next steps will be very memory-consuming:

In [25]:
del bfs_path, distance, categories, graph, connections

As can be seen from above our function is working fine. Now we work on a very very big dictionary. In order to do so we save it chunk by chunk we define the size the save the files for later computation of median.

In [ ]:
node_distance = {}

In [ ]:
s = time.time()
size = 0
for article in C0_list:
    size += 1
    bfs_path = bfs(adj_dict,article,giant_set) # bfs distance on whole list 
    node_distance[article] = bfs_path # a dict of shortest paths
    if (size%300) == 0:
        print(size)        
        f = open("node_distance"+str(size)+".pkl","wb")
        pickle.dump(node_distance,f)
        f.close()
        node_distance = {} # reset our dict
print(time.time()-s)

Get the last items for our dictionary:

In [ ]:
s = time.time()
node_distance = {}
for article in C0_list[3600:]:
    bfs_path = bfs(adj_dict,article,giant_set) # bfs distance on whole list 
    node_distance[article] = bfs_path # a dict of shortest paths
    f = open("node_distance"+str(len(C0_list))+".pkl","wb")
    pickle.dump(node_distance,f)
    f.close()
node_distance = {} # reset our dict
print(time.time()-s)

### As we merged all the list of articles in the previous steps for faster computation of Breadth first search, now we have to create a dictionary to record down the positions of articles for each category

In [29]:
s = time.time()
index = {} # a dictionary records down positions of the articles
for k, v in reduce_categories.items():
    num_list = []
    for i in v:
        num_list.append(giant_set.index(i))
    index[k] = num_list
print(time.time()-s)

2380.6984961032867


Save the file for future usage

In [32]:
f = open("index.pkl","wb")
pickle.dump(index,f)
f.close()

### Now we can find the median and arrange the block

In [ ]:
median_list = []

In [54]:
s = time.time()
files = ['300','600','900','1200','1500','1800',
         '2100','2400','2700','3000','3300','3600','3629'] #get all the files 

for key, value in index.items(): # key gives the list name, value is the indeces
    if key == C0:                # if key is the input
        median_list.append((C0,0.0))
    else:    
        sub_list = []
        for file in files:
            with open("node_distance"+file+".pkl", 'rb') as f:
                node_distance = pickle.load(f)    
                for k, v in node_distance.items(): # v gives distances, k gives articles in input        
                    for i in value:
                        sub_list.append(v[i])
        # calculate median for each category
        median = statistics.median(sub_list)
        component = (key,median)
        median_list.append(component)
print(time.time()-s)

3997.3821783065796


In [213]:
f = open("median_list.pkl","wb")
pickle.dump(median_list,f)
f.close()

In [26]:
with open('median_list.pkl', 'rb') as f:
    median_list = pickle.load(f)

In [27]:
median_list    # take a look

[('English_footballers', inf),
 ('The_Football_League_players', 11.0),
 ('Association_football_forwards', inf),
 ('Association_football_goalkeepers', inf),
 ('Association_football_midfielders', inf),
 ('Association_football_defenders', inf),
 ('Living_people', inf),
 ('Harvard_University_alumni', 15.0),
 ('Major_League_Baseball_pitchers', inf),
 ('Members_of_the_United_Kingdom_Parliament_for_English_constituencies', 7.0),
 ('Indian_films', 7.0),
 ('Year_of_death_missing', 0.0),
 ('Year_of_birth_missing_(living_people)', inf),
 ('Rivers_of_Romania', 8),
 ('Main_Belt_asteroids', inf),
 ('Asteroids_named_for_people', inf),
 ('English-language_albums', 7.0),
 ('British_films', 6.0),
 ('English-language_films', 6.0),
 ('American_films', 6.0),
 ('People_from_New_York_City', 7.0),
 ('American_television_actors', 6),
 ('American_film_actors', 6.0),
 ('Debut_albums', 8.0),
 ('Black-and-white_films', 6.0),
 ('Year_of_birth_missing', inf),
 ('Place_of_birth_missing_(living_people)', 9.0),
 ('Amer

## Block Ranking

In [28]:
median_list.sort(key=itemgetter(1))
median_list

[('Year_of_death_missing', 0.0),
 ('British_films', 6.0),
 ('English-language_films', 6.0),
 ('American_films', 6.0),
 ('American_television_actors', 6),
 ('American_film_actors', 6.0),
 ('Black-and-white_films', 6.0),
 ('Members_of_the_United_Kingdom_Parliament_for_English_constituencies', 7.0),
 ('Indian_films', 7.0),
 ('English-language_albums', 7.0),
 ('People_from_New_York_City', 7.0),
 ('Rivers_of_Romania', 8),
 ('Debut_albums', 8.0),
 ('Place_of_birth_missing_(living_people)', 9.0),
 ('Windows_games', 9.0),
 ('The_Football_League_players', 11.0),
 ('Harvard_University_alumni', 15.0),
 ('English_footballers', inf),
 ('Association_football_forwards', inf),
 ('Association_football_goalkeepers', inf),
 ('Association_football_midfielders', inf),
 ('Association_football_defenders', inf),
 ('Living_people', inf),
 ('Major_League_Baseball_pitchers', inf),
 ('Year_of_birth_missing_(living_people)', inf),
 ('Main_Belt_asteroids', inf),
 ('Asteroids_named_for_people', inf),
 ('Year_of_birt

### Visualize in a dataframe

In [29]:
import pandas as pd
df = pd.DataFrame(median_list,columns= ['Category','Median_Score'])
df

,Category,Median_Score
0,Year_of_death_missing,0.000000
1,British_films,6.000000
2,English-language_films,6.000000
3,American_films,6.000000
4,American_television_actors,6.000000
5,American_film_actors,6.000000
6,Black-and-white_films,6.000000
7,Members_of_the_United_Kingdom_Parliament_for_E...,7.000000
8,Indian_films,7.000000
9,English-language_albums,7.000000


## Sorting the nodes in each category

### Step1: Computing the sub_graph induced by C0

In [30]:
C0_graph = {}
C0_set = set(C0_list)
for article in C0_list:
    temp = set(G.predecessors(article)) # return predecessors of a node in the graph
    score = len(temp.intersection(C0_set))
    C0_graph[article] = score

Now we have C0 sub-graph of format: {article1:score,article2:score....} <br/>
We are now moving to step 2

### Step2:  Extending the graph to the nodes that belong to C1

As we notice, this step is consisted of two sub-steps. First one is to compute the in-edge scores of the nodes in C1, the second one is to compute the total scores by adding the scores in sub-step 1 to the inter-edge scores from nodes in C0.

In [31]:
C1 = median_list[1][0]
C1

'British_films'

In [32]:
C1_list = reduce_categories.get(C1)
C1_list[0:10]

[1056769,
 1056770,
 1056774,
 1245190,
 1064970,
 1040395,
 1056780,
 1056787,
 1056788,
 811030]

In [33]:
# first sub-step
C1_graph = {}
C1_set = set(C1_list)
for article in C1_list:
    temp = set(G.predecessors(article))
    score = len(temp.intersection(C1_set))
    C1_graph[article] = score

In [34]:
# second sub_step
for article, score in C1_graph.items():
    new_score = score
    temp = set(G.predecessors(article))
    predecessors = temp.intersection(C0_set)
    for i in predecessors:
        new_score += C0_graph.get(i)
    C1_graph[article] = new_score

Now we check the ranking of C1 by the highest score first and going down:

In [35]:
sorted_ = sorted(C1_graph.items(), key=itemgetter(1), reverse = True)
sorted_[:10]

[(1041937, 1501),
 (1253712, 15),
 (1253703, 14),
 (1253707, 13),
 (1253709, 13),
 (1253706, 11),
 (1253711, 8),
 (1044136, 8),
 (1061229, 8),
 (1253690, 7)]

This seems a little bit odd when the article 1041937 has such a high score. However when we check the predecessors of 1041937 the case is well-explained:

In [36]:
len(set(G.predecessors(1041937)))

1654

And a lot of that predecessors should come from the same category, such they give many ones to the score of 1041937!

### Step3:  Repeating Step2 up to the last category of the ranking

In [37]:
score_dict = {**C0_graph,**C1_graph} # joining the first two graphs

In [38]:
len(score_dict.keys())

8011

In [39]:
for i in range(2,len(median_list)):
    Ci = median_list[i][0]
    print(Ci)                          # checking the progress
    Ci_list = reduce_categories.get(Ci)
    
    set_dict = set(score_dict.keys())
    # sub-steps
    Ci_graph = {}
    Ci_set = set(Ci_list)
    for article in Ci_list:
        temp = set(G.predecessors(article))
        score = len(temp.intersection(Ci_set))
        predecessors = temp.intersection(set_dict)
        for i in predecessors:
            score += score_dict.get(i)
        Ci_graph[article] = score
    score_dict = {**score_dict,**Ci_graph}

English-language_films
American_films
American_television_actors
American_film_actors
Black-and-white_films
Members_of_the_United_Kingdom_Parliament_for_English_constituencies
Indian_films
English-language_albums
People_from_New_York_City
Rivers_of_Romania
Debut_albums
Place_of_birth_missing_(living_people)
Windows_games
The_Football_League_players
Harvard_University_alumni
English_footballers
Association_football_forwards
Association_football_goalkeepers
Association_football_midfielders
Association_football_defenders
Living_people
Major_League_Baseball_pitchers
Year_of_birth_missing_(living_people)
Main_Belt_asteroids
Asteroids_named_for_people
Year_of_birth_missing
American_military_personnel_of_World_War_II


In [40]:
len(score_dict.keys()) # checking the keys whether it's matched total number of articles or not

422144

Now we check the top articles with the highest scores:

In [41]:
sorted_ = sorted(score_dict.items(), key=itemgetter(1), reverse = True)
sorted_[:10]

[(1400483, 477283325),
 (1400478, 453830224),
 (1061812, 450594335),
 (1400484, 397007898),
 (1061684, 382551609),
 (1062022, 323957782),
 (1163692, 288820905),
 (1061683, 272231405),
 (1163560, 222802986),
 (1061441, 217284790)]

### Now we return the node ranking inside the sub-graph by the name of the articles:

In [42]:
ranking = []
for i in sorted_:
    attribute = (pagenames[i[0]], i[1])
    ranking.extend(attribute)
ranking[:20]

['1400483 Jimmy Carter',
 477283325,
 '1400478 John F. Kennedy',
 453830224,
 '1061812 Paul Newman',
 450594335,
 '1400484 Richard Nixon',
 397007898,
 '1061684 Jack Lemmon',
 382551609,
 '1062022 Robert Altman',
 323957782,
 '1163692 Dean Martin',
 288820905,
 '1061683 Walter Matthau',
 272231405,
 '1163560 Johnny Carson',
 222802986,
 '1061441 Charlton Heston',
 217284790]

### It is quite a striking result for node ranking, considering that we take 'Year_of_death_missing' as the input and now we have pages of famous people as top results !